In [14]:
str_pytket = """
OPENQASM 2.0;
include "qelib1.inc";

qreg qr[2];
creg cr[2];
u2(0.0*pi,1.0*pi) qr[0];
u2(0.0*pi,1.0*pi) qr[1];
cx qr[1],qr[0];
u1(1.74038062947225*pi) qr[0];
cx qr[1],qr[0];
u2(0.0*pi,1.0*pi) qr[0];
u2(0.0*pi,1.0*pi) qr[1];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];
"""

str_qiskit = """
OPENQASM 2.0;
include "qelib1.inc";
qreg qr[2];
creg cr[2];
rxx(5.467567) qr[1],qr[0];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];
"""

In [15]:
from qiskit.qasm2 import loads
from qiskit import qasm2

circuit_qiskit = loads(
    str_qiskit, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
print(circuit_qiskit)
circuit_pytket = loads(
    str_pytket, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
print(circuit_pytket)

      ┌──────────────┐┌─┐   
qr_0: ┤1             ├┤M├───
      │  Rxx(5.4676) │└╥┘┌─┐
qr_1: ┤0             ├─╫─┤M├
      └──────────────┘ ║ └╥┘
cr: 2/═════════════════╩══╩═
                       0  1 
      ┌─────────┐┌───┐┌────────────┐┌───┐┌─────────┐┌─┐   
qr_0: ┤ U2(0,π) ├┤ X ├┤ U1(5.4676) ├┤ X ├┤ U2(0,π) ├┤M├───
      ├─────────┤└─┬─┘└────────────┘└─┬─┘├─────────┤└╥┘┌─┐
qr_1: ┤ U2(0,π) ├──■──────────────────■──┤ U2(0,π) ├─╫─┤M├
      └─────────┘                        └─────────┘ ║ └╥┘
cr: 2/═══════════════════════════════════════════════╩══╩═
                                                     0  1 


In [18]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import Session, Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import pandas as pd


def execute_circuit(qc: QuantumCircuit, platform_name: str):
    # Run the sampler job locally using AerSimulator.
    # Session syntax is supported but ignored.
    aer_sim = AerSimulator()

    # Transpile
    pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
    isa_qc = pm.run(qc)

    # The session is used but ignored in AerSimulator.
    sampler = Sampler(mode=aer_sim)
    result = sampler.run([isa_qc], shots=1000000).result()[0]

    # Section: Results
    counts = result.data.cr.get_counts()
    print(f"Measurement results: {counts}")
    # counts["platform_name"] = platform_name

    return counts


N_TESTS = 5
records_qiskit = []
records_pytket = []

for i in range(N_TESTS):
    print(f"Test {i}")
    print("Qiskit")
    res = execute_circuit(circuit_qiskit, "qiskit")
    records_qiskit.append(res)

    print("Pytket")
    res = execute_circuit(circuit_pytket, "pytket")
    records_pytket.append(res)

df_results_qiskit = pd.DataFrame.from_records(records_qiskit)
df_results_pytket = pd.DataFrame.from_records(records_pytket)

Test 0
Qiskit
Measurement results: {'00': 842491, '11': 157509}
Pytket
Measurement results: {'00': 842731, '11': 157269}
Test 1
Qiskit
Measurement results: {'00': 843610, '11': 156390}
Pytket
Measurement results: {'00': 843078, '11': 156922}
Test 2
Qiskit
Measurement results: {'00': 843333, '11': 156667}
Pytket
Measurement results: {'00': 842539, '11': 157461}
Test 3
Qiskit
Measurement results: {'00': 842397, '11': 157603}
Pytket
Measurement results: {'00': 843726, '11': 156274}
Test 4
Qiskit
Measurement results: {'00': 842425, '11': 157575}
Pytket
Measurement results: {'00': 842741, '11': 157259}


In [17]:
# get the average of the results for each platform
# and compare them

print("Qiskit")
print(df_results_qiskit.mean())

print("Pytket")
print(df_results_pytket.mean())

Qiskit
00    84254.6
11    15745.4
dtype: float64
Pytket
00    84342.2
11    15657.8
dtype: float64


In [20]:
from qiskit.quantum_info import Statevector

str_pytket = """
OPENQASM 2.0;
include "qelib1.inc";

qreg qr[2];
creg cr[2];
u2(0.0*pi,1.0*pi) qr[0];
u2(0.0*pi,1.0*pi) qr[1];
cx qr[1],qr[0];
u1(1.74038062947225*pi) qr[0];
cx qr[1],qr[0];
u2(0.0*pi,1.0*pi) qr[0];
u2(0.0*pi,1.0*pi) qr[1];
"""

str_qiskit = """
OPENQASM 2.0;
include "qelib1.inc";
qreg qr[2];
creg cr[2];
rxx(5.467567) qr[1],qr[0];
"""

circuit_qiskit = loads(
    str_qiskit, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
circuit_pytket = loads(
    str_pytket, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)


sv_qiskit = Statevector.from_instruction(circuit_qiskit)
print(sv_qiskit)

sv_pytket = Statevector.from_instruction(circuit_pytket)
print(sv_pytket)

# compare the statevectors
probabilities_qiskit = sv_qiskit.probabilities_dict()
print(probabilities_qiskit)
probabilities_pytket = sv_pytket.probabilities_dict()
print(probabilities_pytket)

Statevector([-0.91799191+0.j       ,  0.        +0.j       ,
              0.        +0.j       ,  0.        -0.3965991j],
            dims=(2, 2))
Statevector([8.42709152e-01-3.64074768e-01j,
             2.37807546e-17+8.32519166e-17j,
             2.21609759e-18+5.07638576e-17j,
             1.57290848e-01+3.64074768e-01j],
            dims=(2, 2))
{'00': 0.8427091522089964, '11': 0.15729084779100358}
{'00': 0.8427091522089958, '01': 7.496405907426081e-33, '10': 2.5818803232508824e-33, '11': 0.15729084779100347}


In [ ]:
from qiskit.quantum_info import Statevector

sv = Statevector.from_instruction(circuit_qiskit)
print(sv)

In [67]:
from pytket.passes import KAKDecomposition
from pytket.extensions.qiskit import qiskit_to_tk
from pytket.qasm import circuit_to_qasm_str
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

qc = QuantumCircuit(2, 2)
# qc.rzz(0.462419,0, 1)
qc.rxx(5.467567, 1, 0)
# qc.measure(0, 0)
# qc.measure(1, 1)

qc = qc.decompose(reps=2)
# from qiskit import transpile
# qc = transpile(qc, basis_gates=['u1', 'u2', 'cx'])
print(qc)


tket_circ = qiskit_to_tk(qc)
qasm_str_tket = circuit_to_qasm_str(tket_circ, header='qelib1',
                                    maxwidth=200)
print(qasm_str_tket)


# pass_ = KAKDecomposition()
# qc_after = pass_.apply(tket_circ)
# qasm_str_tket = circuit_to_qasm_str(qc_after, header='qelib1',
#                                     maxwidth=200)
# print(qasm_str_tket)

global phase: 3.5494
     ┌─────────┐┌───┐┌────────────┐┌───┐┌─────────┐
q_0: ┤ U2(0,π) ├┤ X ├┤ U1(5.4676) ├┤ X ├┤ U2(0,π) ├
     ├─────────┤└─┬─┘└────────────┘└─┬─┘├─────────┤
q_1: ┤ U2(0,π) ├──■──────────────────■──┤ U2(0,π) ├
     └─────────┘                        └─────────┘
c: 2/══════════════════════════════════════════════
                                                   
OPENQASM 2.0;
include "qelib1.inc";

qreg q[2];
creg c[2];
u2(0.0*pi,1.0*pi) q[0];
u2(0.0*pi,1.0*pi) q[1];
cx q[1],q[0];
u1(1.74038062947225*pi) q[0];
cx q[1],q[0];
u2(0.0*pi,1.0*pi) q[0];
u2(0.0*pi,1.0*pi) q[1];



In [41]:
circuit_single_decompose = loads(
    qasm_str_tket, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
sv_pytket_single_decompose = Statevector.from_instruction(
    circuit_single_decompose)

probabilities_pytket_single_decompose = sv_pytket_single_decompose.probabilities_dict()
print(probabilities_pytket_single_decompose)

{'00': 1.0}


In [53]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import UnitaryGate
from math import sin, cos, pi


theta = 5.467567  # Example angle
matrix = [[cos(theta/2), 0, 0, -1j*sin(theta/2)],
          [0, cos(theta/2), -1j*sin(theta/2), 0],
          [0, -1j*sin(theta/2), cos(theta/2), 0],
          [-1j*sin(theta/2), 0, 0, cos(theta/2)]]
gate = UnitaryGate(matrix, label='rxx')

circuit = QuantumCircuit(2)
circuit.append(gate, [1, 0])

circuit = circuit.decompose(gates_to_decompose={'rxx'})
print(circuit)

sv_unitary_gate = Statevector.from_instruction(circuit)
probabilities_unitary_gate = sv_unitary_gate.probabilities_dict()
print(probabilities_unitary_gate)

     ┌──────┐
q_0: ┤1     ├
     │  rxx │
q_1: ┤0     ├
     └──────┘
{'00': 0.8427091522089964, '11': 0.15729084779100358}


In [68]:
from qiskit import transpile
qc_isa = transpile(circuit, basis_gates=['u1', 'u2', 'cx'])
print(qc_isa)

global phase: 0.2753
     ┌──────────────┐┌──────────┐┌─────────┐┌──────────┐┌──────────┐»
q_0: ┤ U1(-0.56139) ├┤ U1(-π/2) ├┤ U2(0,π) ├┤ U1(-π/2) ├┤ U1(3π/2) ├»
     └──┬───────┬───┘├──────────┤├─────────┤├──────────┤├──────────┤»
q_1: ───┤ U1(0) ├────┤ U1(-π/2) ├┤ U2(0,π) ├┤ U1(-π/2) ├┤ U1(3π/2) ├»
        └───────┘    └──────────┘└─────────┘└──────────┘└──────────┘»
«     ┌──────────┐┌─────────┐┌──────────┐┌────────┐┌───┐ ┌────────────┐ »
«q_0: ┤ U1(-π/2) ├┤ U2(0,π) ├┤ U1(-π/2) ├┤ U1(3π) ├┤ X ├─┤ U1(-2.243) ├─»
«     ├──────────┤├─────────┤├──────────┤├────────┤└─┬─┘┌┴────────────┴┐»
«q_1: ┤ U1(-π/2) ├┤ U2(0,π) ├┤ U1(-π/2) ├┤ U1(3π) ├──■──┤ U1(-0.21152) ├»
«     └──────────┘└─────────┘└──────────┘└────────┘     └──────────────┘»
«     ┌──────────┐┌─────────┐┌──────────┐┌────────────┐┌──────────┐┌─────────┐»
«q_0: ┤ U1(-π/2) ├┤ U2(0,π) ├┤ U1(-π/2) ├┤ U1(4.8546) ├┤ U1(-π/2) ├┤ U2(0,π) ├»
«     ├──────────┤├─────────┤├──────────┤└─┬────────┬─┘├──────────┤├─────────┤»
«q_1: ┤ U1(-π/2) ├┤

Circuit:
<tket::Circuit, qubits=5, gates=5>
